In [3]:
import pandas as pd

In [5]:
import numpy as np

In [7]:
path='C:/Users/user/machinelearningfaundation(washingtonuniversity)/people_wiki.csv'
df=pd.read_csv(path)

In [9]:
df=pd.DataFrame(df)
df.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


In [11]:
df.shape

(59071, 3)

In [19]:
##Explore dataset and checkout the text it contains

In [21]:
obama=df[df['name']=='Barack Obama']

In [23]:
obama

,URI,name,text
35817,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born augu...


In [25]:
##word count function

In [25]:
from collections import Counter
def word_count(text):
    if isinstance(text, str):  # Check if the value is a string
        words = text.split()
        return dict(Counter(words))
    else:
        return {} 

In [27]:
obama['word_count']=obama['text'].apply(word_count)

C:\Users\user\AppData\Local\Temp\ipykernel_2104\2144078428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obama['word_count']=obama['text'].apply(word_count)


In [29]:
obama

,URI,name,text,word_count
35817,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born augu...,"{'barack': 1, 'hussein': 1, 'obama': 9, 'ii': ..."


In [31]:
Alfred=df[df['name']=='Alfred J. Lewy']
Alfred

,URI,name,text
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...


In [33]:
obama_word_count = obama['word_count'].apply(pd.Series).stack().reset_index()

In [35]:
obama_word_count

,level_0,level_1,0
0,35817,barack,1
1,35817,hussein,1
2,35817,obama,9
3,35817,ii,1
4,35817,brk,1
...,...,...,...
268,35817,operations,1
269,35817,sought,1
270,35817,normalize,1
271,35817,relations,1


In [37]:
obama_word_count.columns = ['index', 'word', 'count']

# Drop unnecessary index column if needed
obama_word_count = obama_word_count[['word', 'count']]

In [39]:
obama_word_count_sorted = obama_word_count.sort_values(by='count', ascending=False)

In [43]:
obama_word_count_sorted.head(10)

,word,count
12,the,40
26,in,30
14,and,21
17,of,18
23,to,14
47,his,11
2,obama,9
139,act,8
37,he,7
29,a,7


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Initialize the TfidfVectorizer with limited features
vectorizer = TfidfVectorizer()

# Apply TF-IDF to the 'text' column
tfidf_sparse_matrix = vectorizer.fit_transform(df['text'])

# Keep it as sparse matrix (do not convert to dense)
print(tfidf_sparse_matrix.shape)  # Output: (59071, 10000)



(59071, 548429)


In [53]:
v = TfidfVectorizer()
t_output=v.fit_transform(df['text'])
# Get the vocabulary


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [55]:
vocab = v.vocabulary_

# Print the first 10 items in the vocabulary
for word, index in list(vocab.items())[:10]:
    print(f"{word}: {index}")

digby: 160652
morrell: 336162
born: 96612
10: 704
october: 359577
1979: 11143
is: 259893
former: 203171
australian: 72887
rules: 427313


In [73]:
feature_names = v.get_feature_names_out()

# Create a dictionary of TF-IDF values for each row
tfidf_dicts = []
for row in t_output:
    tfidf_dict = {feature_names[col]: row[0, col] for col in row.nonzero()[1]}
    tfidf_dicts.append(tfidf_dict)

# Add the dictionaries as a new column to the DataFrame
df['tfidf'] = tfidf_dicts


In [75]:
df.head()

,URI,name,text,tfidf
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{'melbourne': 0.04943650649482413, 'college': ..."
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{'every': 0.03831773682944378, 'capsule': 0.07..."
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{'society': 0.03991955990922275, 'hamilton': 0..."
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{'kurdlawitzpreis': 0.08928312306619517, 'spec..."
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{'curtis': 0.052582727709648204, 'promo': 0.06..."


In [77]:
Alfred=df[df['name']=='Alfred J. Lewy']
Alfred['tfidf']

1    {'every': 0.03831773682944378, 'capsule': 0.07...
Name: tfidf, dtype: object

In [83]:
Obama=df[df['name']=='Barack Obama']
clinton=df[df['name']=='Bill Clinton']
beckham=df[df['name']=='David Beckham']

In [79]:
Alfred['sorted_tfidf'] = Alfred['tfidf'].apply(lambda x: dict(sorted(x.items(), key=lambda item: item[1], reverse=True)))

# Display the sorted TF-IDF dictionary
print(Alfred[['name', 'sorted_tfidf']].head())

             name                                       sorted_tfidf
1  Alfred J. Lewy  {'melatonin': 0.47165369680859387, 'lewy': 0.2...


C:\Users\user\AppData\Local\Temp\ipykernel_9692\1157993728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Alfred['sorted_tfidf'] = Alfred['tfidf'].apply(lambda x: dict(sorted(x.items(), key=lambda item: item[1], reverse=True)))


In [98]:

from sklearn.metrics.pairwise import cosine_similarity

# Extract TF-IDF dictionaries for Obama and Clinton
obama_tfidf = Obama['tfidf'].iloc[0]
clinton_tfidf = clinton['tfidf'].iloc[0]
print(obama_tfidf)


35817    {'2012obama': 0.07885433263257774, 'doddfrank'...
Name: tfidf, dtype: object


In [106]:
## cosine distance using raw text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Get the raw text data for Obama and Clinton
obama_text = Obama['text'].iloc[0]
clinton_text = clinton['text'].iloc[0]

# Use TfidfVectorizer to transform the raw text into TF-IDF vectors
vectorizer = TfidfVectorizer()

# Combine the raw texts of Obama and Clinton into a list
texts = [obama_text, clinton_text]

# Apply the tf-idf transformation
tfidf_matrix = vectorizer.fit_transform(texts)

# Compute cosine similarity between the two vectors (Obama and Clinton)
similarity_obama_clinton = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

print("Cosine Similarity between Obama and Clinton:", similarity_obama_clinton[0][0])


Cosine Similarity between Obama and Clinton: 0.734728298394171


In [152]:
#similarity using tfidf
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.spatial.distance import cosine


# Step 1: Create the word count matrix and compute TF-IDF
count_vectorizer = CountVectorizer(stop_words='english')
word_count_matrix = count_vectorizer.fit_transform(df['text'])

tfidf_transformer = TfidfTransformer()
tfidf_matrix = tfidf_transformer.fit_transform(word_count_matrix)

# Step 2: Function to get sorted TF-IDF values for a specific person
def get_sorted_tfidf(name):
    # Find the index of the person in the DataFrame
    person_index = df[df['name'] == name].index[0]
    
    # Extract the person's row from the TF-IDF matrix
    tfidf_row = tfidf_matrix[person_index]
    
    # Convert the sparse row to a dense format
    tfidf_values = tfidf_row.toarray().flatten()
    
    # Create a DataFrame with words and their corresponding TF-IDF values
    tfidf_data = pd.DataFrame({
        'word': count_vectorizer.get_feature_names_out(),
        'tfidf': tfidf_values
    })
    
    # Sort the DataFrame by TF-IDF values in descending order
    tfidf_data = tfidf_data.sort_values(by='tfidf', ascending=False)
    return tfidf_data

# Step 3: Compute and display TF-IDF values for each person
obama_tfidf = get_sorted_tfidf('Barack Obama')
clooney_tfidf = get_sorted_tfidf('George Clooney')
clinton_tfidf = get_sorted_tfidf('Bill Clinton')
beckham_tfidf = get_sorted_tfidf('David Beckham')

# Print sorted TF-IDF data for each person
print("Obama's Sorted TF-IDF:\n", obama_tfidf.head())
print("\nClooney's Sorted TF-IDF:\n", clooney_tfidf.head())
print("\nClinton's Sorted TF-IDF:\n", clinton_tfidf.head())
print("\nBeckham's Sorted TF-IDF:\n", beckham_tfidf.head())

# Step 4: Compute cosine similarity between Obama and Clinton
obama_index = df[df['name'] == 'Barack Obama'].index[0]
clinton_index = df[df['name'] == 'Bill Clinton'].index[0]

# Access their TF-IDF vectors directly from the sparse matrix
obama_vector = tfidf_matrix[obama_index].toarray().flatten()
clinton_vector = tfidf_matrix[clinton_index].toarray().flatten()

# Compute cosine distance (1 - cosine similarity)
cosine_distance = cosine(obama_vector, clinton_vector)
print("\nCosine distance between Obama and Clinton's TF-IDF vectors:", cosine_distance)


Obama's Sorted TF-IDF:
            word     tfidf
358378    obama  0.413495
45070       act  0.282170
259085     iraq  0.171970
292368      law  0.163903
138523  control  0.149369

Clooney's Sorted TF-IDF:
                word     tfidf
127854      clooney  0.350758
490967     thriller  0.241411
168308        drama  0.179177
45486         actor  0.161696
132236  comedydrama  0.152711

Clinton's Sorted TF-IDF:
              word     tfidf
127657    clinton  0.521952
228969      haiti  0.174967
395397  president  0.168248
272791      kappa  0.162194
65699    arkansas  0.157896

Beckham's Sorted TF-IDF:
            word     tfidf
83844   beckham  0.618249
209796   galaxy  0.199990
196499     fifa  0.163539
386825   player  0.149617
293372   league  0.147291

Cosine distance between Obama and Clinton's TF-IDF vectors: 0.8110328230075754


In [154]:
obama_index

35817

In [156]:
obama_vector

array([0., 0., 0., ..., 0., 0., 0.])

In [162]:
o_v=tfidf_matrix[obama_index].toarray().flatten()
c_v = tfidf_matrix[clinton_index].toarray().flatten()

In [164]:
cosine_distance = cosine(o_v, c_v)
print("\nCosine distance between Obama and Clinton's TF-IDF vectors:", cosine_distance)


Cosine distance between Obama and Clinton's TF-IDF vectors: 0.674977752652465


In [166]:
word_count_matrix

<59071x548115 sparse matrix of type '<class 'numpy.int64'>'
	with 8078359 stored elements in Compressed Sparse Row format>

In [168]:
print(word_count_matrix)

  (0, 160586)	1
  (0, 336006)	5
  (0, 96562)	1
  (0, 704)	1
  (0, 359398)	1
  (0, 11143)	1
  (0, 72855)	3
  (0, 427109)	1
  (0, 202134)	1
  (0, 386788)	3
  (0, 272389)	3
  (0, 111892)	2
  (0, 202093)	10
  (0, 293372)	4
  (0, 48412)	1
  (0, 529065)	1
  (0, 72702)	1
  (0, 172590)	1
  (0, 442626)	3
  (0, 528963)	2
  (0, 380705)	1
  (0, 34683)	1
  (0, 111154)	1
  (0, 191018)	1
  (0, 461663)	1
  :	:
  (59070, 104364)	1
  (59070, 475179)	1
  (59070, 138816)	1
  (59070, 180342)	1
  (59070, 374265)	1
  (59070, 152743)	1
  (59070, 138553)	1
  (59070, 386115)	1
  (59070, 371604)	1
  (59070, 307073)	1
  (59070, 257042)	1
  (59070, 156823)	1
  (59070, 266377)	2
  (59070, 336416)	1
  (59070, 22951)	1
  (59070, 250788)	1
  (59070, 126884)	1
  (59070, 418921)	1
  (59070, 413994)	1
  (59070, 193180)	1
  (59070, 135411)	1
  (59070, 149753)	4
  (59070, 193198)	1
  (59070, 149752)	1
  (59070, 502085)	1


In [170]:
o_v=tfidf_sparse_matrix[obama_index].toarray().flatten()
c_v = tfidf_sparse_matrix[clinton_index].toarray().flatten()

In [172]:
cosine_distance = cosine(o_v, c_v)
print("\nCosine distance between Obama and Clinton's TF-IDF vectors:", cosine_distance)


Cosine distance between Obama and Clinton's TF-IDF vectors: 0.6433967902533614


In [174]:
df.head()

,URI,name,text,tfidf
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{'melbourne': 0.04943650649482413, 'college': ..."
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{'every': 0.03831773682944378, 'capsule': 0.07..."
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{'society': 0.03991955990922275, 'hamilton': 0..."
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{'kurdlawitzpreis': 0.08928312306619517, 'spec..."
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{'curtis': 0.052582727709648204, 'promo': 0.06..."


#applying knn(after 41 cell one can do this)

In [47]:
#splitting data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.text, 
    df.name, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
   
)

In [69]:
X=df['text']
y=df['name']

In [71]:
#knn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('KNN', KNeighborsClassifier())         
])

#2. fit with X_train and y_train
clf.fit(X,y)


#3. get the predictions for X_test and store it in y_pred
#y_pred = clf.predict(X_test)


#4. print the classfication report
#print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('KNN', KNeighborsClassifier())])

In [51]:
print(X_test[:5])

12912    joe williams qso is a cook islands politician ...
51084    corrado grabbi born 29 july 1975 in turin is a...
45100    thomas tom amadeus kalil born october 4 1963 i...
37065    hikmet temel akarsu born 1960 in gmhane turkey...
20752    christopher james chris kluwe klui born decemb...
Name: text, dtype: object


In [53]:
print(y_test[:5])

12912    Joe Williams (Cook Islands politician)
51084                            Corrado Grabbi
45100                              Thomas Kalil
37065                       Hikmet Temel Akarsu
20752                               Chris Kluwe
Name: name, dtype: object


In [55]:
print(y_pred[:5])

['Edison James' 'Jason Roberts (footballer)' 'Abbe Mowshowitz'
 'Ali F. Bilir' 'Bill Brown (American football)']


In [59]:
index_value = df.loc[df['name'] == 'Barack Obama'].index[0]
print(index_value)

35817


In [77]:
# 1. Get the vectorized representation of X_test
vectorizer = clf.named_steps['vectorizer_tfidf']  # Extract the TfidfVectorizer
X_test_vectorized = vectorizer.transform(X)

# 2. Use the trained KNeighborsClassifier to find the 5 nearest neighbors
knn_model = clf.named_steps['KNN']  # Extract the KNeighborsClassifier



In [79]:
distances, indices = knn_model.kneighbors(X_test_vectorized[35817], n_neighbors=5)

# 3. Print the indices and distances of the 5 nearest neighbors
print("Indices of nearest neighbors:", indices)
print("Distances of nearest neighbors:", distances)

# 4. If you want the actual text and names corresponding to these indices:
nearest_indices = indices[0]  # Flatten the array
print("Nearest texts:")
print(X.iloc[nearest_indices])
print("Nearest names:")
print(y.iloc[nearest_indices])

Indices of nearest neighbors: [[35817 24478 57108 38376 38714]]
Distances of nearest neighbors: [[0.         1.06843875 1.1098956  1.11802815 1.13996938]]
Nearest texts:
35817    barack hussein obama ii brk husen bm born augu...
24478    joseph robinette joe biden jr dosf rbnt badn b...
57108    hillary diane rodham clinton hlri dan rdm klnt...
38376    samantha power born september 21 1970 is an ir...
38714    eric stern is the director of operations for t...
Name: text, dtype: object
Nearest names:
35817               Barack Obama
24478                  Joe Biden
57108     Hillary Rodham Clinton
38376             Samantha Power
38714    Eric Stern (politician)
Name: name, dtype: object
